# First Iteration Code Development and Testing

## Algorithmic Trading - Applying Moving Average

### Importing Libraries and Dependencies

In [446]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import warnings
warnings.simplefilter(action="ignore")

In [447]:
# Input Parameters

# Window runs: Short Term (9-21, 21-50, 9-50)
# Long Term: 50-100, 21-100, Golden Cross - Death Cross: 50-200
# Objective: Which of the above combinations gives the best return for each stock

# Set the short window and long windows
short_window = 50
long_window = 200

# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

### Importing Source CSV as Dataframe

In [448]:
# Reading CSV
# Set the file path
filepath = Path("../Resources/BTC-USD.csv")

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index and auto-format the datetime string
temp_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)
btc_df = temp_df.dropna()

# Print the DataFrame
btc_df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-06,3836.519043,4093.297363,3826.513184,4076.632568,4076.632568,5597027439
1,2019-01-07,4078.584961,4092.613525,4020.894043,4025.248291,4025.248291,5228625637
2,2019-01-08,4028.472168,4109.020996,3996.955322,4030.847900,4030.847900,5306593305
3,2019-01-09,4031.552002,4068.403564,4022.662842,4035.296387,4035.296387,5115905224
4,2019-01-10,4034.411377,4064.066650,3659.174561,3678.924561,3678.924561,6874143795
5,2019-01-11,3674.015381,3713.881836,3653.069824,3687.365479,3687.365479,5538712864
6,2019-01-12,3686.973145,3698.978271,3653.810791,3661.301025,3661.301025,4778170882
7,2019-01-13,3658.868164,3674.760010,3544.927246,3552.953125,3552.953125,4681302465
8,2019-01-14,3557.311035,3727.836182,3552.285156,3706.052246,3706.052246,5651384489
9,2019-01-15,3704.216309,3720.153320,3619.949219,3630.675293,3630.675293,5537192301


### Generating Moving Average Crossover Signal

In [449]:
# Grab just the `date` and `close` from the dataset
signals_df = aapl_df.loc[:, ["Date", "Close"]].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA200"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0


signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA200"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA50,SMA200,Signal,Entry/Exit
Date,,,,,
2019-01-06,4076.632568,NaN,NaN,0.0,NaN
2019-01-07,4025.248291,NaN,NaN,0.0,0.0
2019-01-08,4030.847900,NaN,NaN,0.0,0.0
2019-01-09,4035.296387,NaN,NaN,0.0,0.0
2019-01-10,3678.924561,NaN,NaN,0.0,0.0
2019-01-11,3687.365479,NaN,NaN,0.0,0.0
2019-01-12,3661.301025,NaN,NaN,0.0,0.0
2019-01-13,3552.953125,NaN,NaN,0.0,0.0
2019-01-14,3706.052246,NaN,NaN,0.0,0.0


 ### Plotting Entry/Exit Points of Moving Average Crossover Trading Strategy

In [450]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA200']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

In [451]:
# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

### Backtest the Trading Strategy

In [452]:
# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df["Position"] = share_size * signals_df["Signal"]

# Find the points in time where a 500 share position is bought or sold
signals_df["Entry/Exit Position"] = signals_df["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df["Portfolio Holdings"] = (
    signals_df["Close"] * signals_df["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df["Portfolio Cash"] = (
    initial_capital - (signals_df["Close"] * signals_df["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df["Portfolio Total"] = (
    signals_df["Portfolio Cash"] + signals_df["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
signals_df["Portfolio Daily Returns"] = signals_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
signals_df["Portfolio Cumulative Returns"] = (
    1 + signals_df["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
signals_df.tail()

,Close,SMA50,SMA200,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2021-01-02,32127.267578,20932.040508,13598.112334,1.0,0.0,500.0,0.0,1.606363e+07,-6.200519e+06,9.863115e+06,0.162204,97.631145
2021-01-03,32782.023438,21266.318203,13714.621177,1.0,0.0,500.0,0.0,1.639101e+07,-6.200519e+06,1.019049e+07,0.033192,100.904924
2021-01-04,31971.914063,21586.644727,13827.421543,1.0,0.0,500.0,0.0,1.598596e+07,-6.200519e+06,9.785438e+06,-0.039748,96.854377
2021-01-05,33992.429688,21932.171094,13950.943599,1.0,0.0,500.0,0.0,1.699621e+07,-6.200519e+06,1.079570e+07,0.103241,106.956956
2021-01-06,36824.363281,22315.750234,14088.403711,1.0,0.0,500.0,0.0,1.841218e+07,-6.200519e+06,1.221166e+07,0.131160,121.116624


### Plot Entry/Exit Points of Backtest Results

In [453]:
# Visualize exit position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

### Prepare Portfolio Evaluation Metrics DataFrame

In [454]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [455]:
### Calculate and Assign Portfolio Evaluation Metrics

In [456]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']]
sortino_ratio_df.loc[:,'Downside Returns'] = 0
sortino_ratio_df.head()

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

,Backtest
Annual Return,-34.3158
Cumulative Returns,121.117
Annual Volatility,62.9886
Sharpe Ratio,-0.544793
Sortino Ratio,-0.547882


### Prepare Trade Evaluation Metrics DataFrame

In [457]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [458]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss =  exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2019-07-24,2019-10-26,500.0,9811.925781,9244.972656,4.905963e+06,4622486.328,-2.834766e+05
1,AAPL,2020-02-19,2020-03-25,500.0,9633.386719,6681.062988,4.816693e+06,3340531.494,-1.476162e+06


In [459]:
### Create Hvplot Line Chart of Closing, SMA50, and SMA100

In [460]:
price_df = signals_df[['Close', 'SMA50', 'SMA100']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

KeyError: "['SMA100'] not in index"

In [ ]:
### Create Hvplot Table of Portfolio Metrics

In [ ]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

In [ ]:
### Create Hvplot Table of Trade Metrics

In [ ]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table